In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
import hvplot.xarray
import cartopy.crs as crs

In [29]:
from src.dataloader import *

In [30]:
ds_train = TiggeMRMSDataset(
    tigge_dir='/datadrive/tigge/32km/',
    tigge_vars=['total_precipitation'],
    mrms_dir='/datadrive/mrms/4km/RadarOnly_QPE_06H/',
    rq_fn='/datadrive/mrms/4km/RadarQuality.nc',
#     const_fn='/datadrive/tigge/32km/constants.nc',
#     const_vars=['orog', 'lsm'],
    val_days=7,
    split='train'
)

In [31]:
ds_valid = TiggeMRMSDataset(
    tigge_dir='/datadrive/tigge/32km/',
    tigge_vars=['total_precipitation'],
    mrms_dir='/datadrive/mrms/4km/RadarOnly_QPE_06H/',
    rq_fn='/datadrive/mrms/4km/RadarQuality.nc',
#     const_fn='/datadrive/tigge/32km/constants.nc',
#     const_vars=['orog', 'lsm'],
    val_days=7,
    split='valid',
    mins=ds_train.mins,
    maxs=ds_train.maxs
)

In [32]:
len(ds_train), len(ds_valid)

(5328, 2016)

In [33]:
sampler = torch.utils.data.WeightedRandomSampler(ds_train.compute_weights(), len(ds_train))

In [34]:
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=32, sampler=sampler)
dl_valid = torch.utils.data.DataLoader(ds_valid, batch_size=32)

In [35]:
X, y = next(iter(dl_train))

In [36]:
X.shape, y.shape

(torch.Size([32, 1, 16, 16]), torch.Size([32, 1, 128, 128]))

In [48]:
import torch.nn as nn
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning.trainer.trainer import Trainer
from torch.nn import functional as F

In [52]:
class UpscalingCNN(LightningModule):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(16, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(16, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU()
        )
        self.conv4 = nn.Sequential(
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(16, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU()
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(16, 1, kernel_size=5, stride=1, padding=2)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        out = self.conv5(x)
        return out
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)
        

In [53]:
model = UpscalingCNN()

In [54]:
model(X).shape

torch.Size([32, 1, 128, 128])

In [55]:
trainer = Trainer()

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [56]:
trainer.fit(model, dl_train)


  | Name  | Type       | Params
-------------------------------------
0 | conv1 | Sequential | 416   
1 | conv2 | Sequential | 6.4 K 
2 | conv3 | Sequential | 6.4 K 
3 | conv4 | Sequential | 6.4 K 
4 | conv5 | Sequential | 401   
-------------------------------------
20.1 K    Trainable params
0         Non-trainable params
20.1 K    Total params


Epoch 0:   0%|          | 0/167 [00:00<?, ?it/s] 

/home/stephan/.conda/envs/stephan/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 2:  16%|█▌        | 27/167 [00:21<01:52,  1.25it/s, loss=0.00276, v_num=0] 


/home/stephan/.conda/envs/stephan/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [62]:
trainer.logger.dd

TypeError: agg_and_log_metrics() missing 1 required positional argument: 'metrics'

In [20]:
criterion = nn.MSELoss()

In [24]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [26]:
for X, y in dl_train:
#     outputs = model(X)
#     loss = criterion(outputs, y)
    
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    print(loss)

tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBac

KeyboardInterrupt: 